> **Note:** if you are running this notebook on Google Colab, the next cell will install quantumsymmetry and its dependencies:

In [ ]:
if 'google.colab' in str(get_ipython()):
    %%capture
    !pip -q install quantumsymmetry
    import quantumsymmetry

# Running a variational algorithm with a symmetry-adapted encoding

## The unitary coupled clusters with singles and doubles (UCCSD) circuit with a symmetry-adapted encoding

We can then use `quantumsymmetry` to construct a circuit for the most common variational ansatz at the time of writing, unitary coupled clusters with singles and doubles (UCCSD).

An example code for the $H_3^+$ dimer in a STO-3G basis is given below.

In [ ]:
from quantumsymmetry import Encoding
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

In [ ]:
#Parameters for H3+ in the sto-3g basis
atom = 'H 0 0.377 0; H -0.435 -0.377 0; H 0.435 -0.377 0'
charge = 1
spin = 0
basis = 'sto3g'

We first simulate a VQE run under a UCCSD ansatz with the Jordan-Wigner encoding:

In [ ]:
driver = PySCFDriver(
    atom= atom,
    unit=DistanceUnit.ANGSTROM,
    charge=charge,
    spin=spin,
    basis=basis)

problem = driver.run()

mapper = JordanWignerMapper()
initial_state = HartreeFock(
    num_spatial_orbitals = problem.num_spatial_orbitals,
    num_particles = problem.num_particles,
    qubit_mapper = mapper
    )

ansatz = UCCSD(
    num_spatial_orbitals = problem.num_spatial_orbitals,
    num_particles = problem.num_particles,
    qubit_mapper = mapper,
    initial_state = initial_state
    )

vqe = VQE(
    estimator = Estimator(),
    ansatz = ansatz,
    optimizer = SLSQP()
)

solver = GroundStateEigensolver(mapper, vqe)
vqe_result = solver.solve(problem)

energy1 = vqe_result.total_energies[0]
qubits1 = ansatz.num_qubits
depth1 = ansatz.decompose().decompose().decompose().decompose().depth()


<ipython-input-8-412c3237c664>:25: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator(),


In [ ]:
print(f'The VQE converged to an energy of {energy1} Ha.\nThe circuit depth for the UCCSD ansatz circuit is {depth1} gates on {qubits1} qubits.')

The VQE converged to an energy of -1.2613444947085288 Ha.
The circuit depth for the UCCSD ansatz circuit is 414 gates on 6 qubits.


We then repeat the same VQE UCCSD simulation using a symmetry-adapted encoding and using the `.mapper` property of the `Encoding` object to obtain a Qiskit `QubitMapper` object, and the `.HF_circuit` property to obtain a `QuantumCircuit` to prepare the Hartree-Fock state:

In [ ]:
encoding = Encoding(atom = atom, basis = basis, charge = charge, spin = spin, output_format = 'qiskit')
mapper = encoding.qiskit_mapper
initial_state = encoding.HF_circuit

ansatz = UCCSD(
    num_spatial_orbitals = problem.num_spatial_orbitals,
    num_particles = problem.num_particles,
    qubit_mapper = mapper,
    initial_state = initial_state
    )

vqe = VQE(
    estimator = Estimator(),
    ansatz = ansatz,
    optimizer = SLSQP()
)

solver = GroundStateEigensolver(mapper, vqe)
vqe_result = solver.solve(problem)

energy2 = vqe_result.total_energies[0]
qubits2 = ansatz.num_qubits
depth2 = ansatz.decompose().decompose().decompose().decompose().depth()

<ipython-input-11-a85034e8f9a2>:13: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator(),


In [ ]:
print(f'The VQE converged to an energy of {energy2} Ha.\nThe circuit depth for the UCCSD ansatz circuit is {depth2} gates on {qubits2} qubits.')

The VQE converged to an energy of -1.2613447641572006 Ha.
The circuit depth for the UCCSD ansatz circuit is 80 gates on 3 qubits.


We see that both VQE runs converge to the same energy (up to a small difference in the order of magnitude of $10^{-7}$ Ha). However, by using the symmetry-adapted encoding the number of qubits is reduced from 6 to 3 qubits, and the UCCSD circuit depth is reduced from 414 gates to 80 gates.

<p style="text-align: left"> <a href="04_fermionic_operators.ipynb" />< Previous: Encoding fermionic operators</a> </p>